# 0. 라이브러리 임포트

In [ ]:
import numpy
import pandas

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 1. 시각화 관련 설정

In [ ]:
%matplotlib inline
plt.rcParams['font.size'] = 20
plt.rcParams["figure.figsize"] = (50, 30)
plt.rcParams['lines.linewidth'] = 2
plt.rcParams["axes.grid"] = True

In [ ]:
%%html
<style>
    table { display: inline-block }
    .rendered_html td, .rendered_html th { text-align: left; }
</style>

# 2. 데이터 로딩

In [ ]:
df_base = pandas.read_csv('data-0201-base.csv')
display(df_base)

# 3. 10개 종목 종가에 대한 포트폴리오 비중 데이터 생성

In [ ]:
ASSET_COUNT = 10

import warnings
warnings.filterwarnings('ignore')

window_size = 490
start_date = 20030101

df_mpt_base = df_base.copy()
df_days = df_mpt_base[df_mpt_base['기준일자'] > start_date]
list_days = df_days['기준일자'].tolist()
list_days.sort()

list_cols_mpt = ['기준일자']
for col_name in df_mpt_base.columns:
    if '종가' in col_name:
        list_cols_mpt.append(col_name)
df_mpt = df_mpt_base[list_cols_mpt]
del df_mpt['KOSPI지수-종가']
display(df_mpt)

df_mpt_merged = pandas.DataFrame()
for day in list_days:
    print('종가(KOSPI지수 제외)의 포트폴리오 배분 데이터 생성', day)
    df_part = df_mpt[df_mpt['기준일자'] < day]
    df_part = df_part.tail(490)
    df_part = df_part.set_index(['기준일자'])

    returns_daily = df_part.pct_change()
    returns_annual = returns_daily.mean() * window_size

    cov_daily = returns_daily.cov()
    cov_annual = cov_daily * window_size

    list_port_returns = list()
    list_port_volatility = list()
    list_sharpe_ratio = list()
    list_stock_weights = list()

    num_portfolios = 50000
    numpy.random.seed(524)

    for _ in range(num_portfolios):
        weights = numpy.random.random(ASSET_COUNT)
        weights /= numpy.sum(weights)
        returns = numpy.dot(weights, returns_annual)
        volatility = numpy.sqrt(numpy.dot(weights.T, numpy.dot(cov_annual, weights)))

        sharpe = returns / volatility
        list_sharpe_ratio.append(sharpe)
        list_port_returns.append(returns)
        list_port_volatility.append(volatility)

        list_stock_weights.append(weights)

    portfolio = {
        'Returns': list_port_returns,
        'Volatility': list_port_volatility,
        'Sharpe Ratio': list_sharpe_ratio,
    }
    for counter, symbol in enumerate(cov_annual.columns):
        portfolio[symbol.replace('-price_close', '')] = [Weight[counter] for Weight in list_stock_weights]
    df = pandas.DataFrame(portfolio)
    df_ratio = df.loc[df['Sharpe Ratio'] == numpy.max(df['Sharpe Ratio'])]
    df_ratio['TYPE'] = 'A'
    df_ratio['기준일자'] = day
    df_mpt_merged = pandas.concat([df_mpt_merged, df_ratio], sort=False)
    df_ratio = df.loc[df['Volatility'] == numpy.min(df['Volatility'])]
    df_ratio['TYPE'] = 'B'
    df_ratio['기준일자'] = day
    df_mpt_merged = pandas.concat([df_mpt_merged, df_ratio], sort=False)

display(df_mpt_merged)

In [ ]:
df_mpt_merged_A = df_mpt_merged[df_mpt_merged['TYPE'] == 'A']
df_mpt_merged_A = df_mpt_merged_A[[
                            '기준일자',
                            'Returns',
                            'Volatility',
                            'Sharpe Ratio',
                            'A005930-삼성전자-종가',
                            'A000660-SK하이닉스-종가',
                            'A006400-삼성SDI-종가',
                            'A005380-현대차-종가',
                            'A000270-기아-종가',
                            'A005490-POSCO홀딩스-종가',
                            'A012330-현대모비스-종가',
                            'A015760-한국전력-종가',
                            'A033780-KT&G-종가',
                            'A003550-LG-종가'
                            ]]
df_mpt_merged_A.to_csv('data-0202-mpt-A.csv', index=None)

In [ ]:
df_mpt_merged_B = df_mpt_merged[df_mpt_merged['TYPE'] == 'B']
df_mpt_merged_B = df_mpt_merged_B[[
                            '기준일자',
                            'Returns',
                            'Volatility',
                            'Sharpe Ratio',
                            'A005930-삼성전자-종가',
                            'A000660-SK하이닉스-종가',
                            'A006400-삼성SDI-종가',
                            'A005380-현대차-종가',
                            'A000270-기아-종가',
                            'A005490-POSCO홀딩스-종가',
                            'A012330-현대모비스-종가',
                            'A015760-한국전력-종가',
                            'A033780-KT&G-종가',
                            'A003550-LG-종가'
                            ]]
df_mpt_merged_B.to_csv('data-0202-mpt-B.csv', index=None)